### Views: Keys, Values and Items

#### Views are not Static

These view objects are not static - so it's not like Python makes a copy of the keys, values or items, and uses these static copies. They are like windows (views) into the **current** state of the dictionary. If the dictionary changes, then these views reflect those changes immediately.

Basically these views provide methods that access the underlying dictionary. They do not "own" any data.

In [1]:
d = {'a': 1, 'b': 2}

In [2]:
keys = d.keys()
values = d.values()
items = d.items()

In [3]:
print(id(keys), id(values), id(items))

4347984680 4347985016 4347985064


In [4]:
print(keys)
print(values)
print(items)

dict_keys(['a', 'b'])
dict_values([1, 2])
dict_items([('a', 1), ('b', 2)])


In [5]:
d['z'] = 100

In [6]:
print(id(keys), id(values), id(items))

4347984680 4347985016 4347985064


As you can see the memory address of these view objects has not changed:

In [7]:
print(keys)
print(values)
print(items)

dict_keys(['a', 'b', 'z'])
dict_values([1, 2, 100])
dict_items([('a', 1), ('b', 2), ('z', 100)])


but the view 'contents' have changed. These views are **dynamic**.

#### Mutating a dictionary while iterating over these views

Because these views instantly reflect any modifications made to the underlying dictionary, we have to be careful changing the dictionary while we iterate over a view! 

In [8]:
d = {'a': 1, 'b': 2, 'c': 3}

In [9]:
for k, v in d.items():
    print(k, v)
    del d[k]

a 1


RuntimeError: dictionary changed size during iteration

As you can see Python complains about this. But the interesting thing is that Python does not complain about the deletion itself - notice where the exception occurs - at the loop, not the delete statement.

In fact, the dictionary **has** changed:

In [10]:
d

{'b': 2, 'c': 3}

As you can see the key `a` is gone.

So the deletion happens just fine, but when Python continues the loop, at that point it detects that the dictionary has changed - and an exception is raised at that point. But notice the exception message - Python is complaining about the **size** of the dictionary changing... 
We'll come back to that point in a minute.

What about insertions, will Python complain about it?

In [11]:
d = {'a': 1, 'b': 2, 'c': 3}
for k, v in d.items():
    print(k, v)
    d['z'] = 100

a 1


RuntimeError: dictionary changed size during iteration

No, that's not allowed either.

It is perfectly fine to modify the values though:

In [12]:
d = {'a': 1, 'b': 2, 'c': 3}
for k, v in d.items():
    print(k, v)
    d[k] = 1000

a 1
b 2
c 3


and of course our dictionary values have changed:

In [13]:
d

{'a': 1000, 'b': 1000, 'c': 1000}

What about the other views, are they more tolerant of underlying mutations? We would not expect the key view to allow this, but what about the values view? After all it is not referencing the keys at all...

In [14]:
d = {'a': 1, 'b': 2, 'c': 3}
for v in d.values():
    print(v)
    del d['a']

1


RuntimeError: dictionary changed size during iteration

No, not allowed either! We just cannot change the size of the dictionary (and hence the size of the view too) while iterating over it.

So, if that's the limitation, then we should be able to modify the values of elements as we iterate over the keys:

In [15]:
d = {'a': 1, 'b': 2, 'c': 3}

In [16]:
for key in d.keys():
    d[key] = 100

In [17]:
d

{'a': 100, 'b': 100, 'c': 100}

Even this will work fine:

In [18]:
for k, v in d.items():
    d[k] = v * 2

In [19]:
d

{'a': 200, 'b': 200, 'c': 200}

The moral here is that you should not manipulate the keys of a dictionary as you iterate over it - either directly, or using the views.
Mutating associated values is perfectly fine.

#### Iterating over a dictionary vs iterating over the keys view

As we just mentioned, dictionaries implement the iterable protocol which iterates over the keys of the dictionary:

In [20]:
d = dict.fromkeys('python', 0)

In [21]:
for k in d:
    print(k)

p
y
t
h
o
n


This would be the same as requesting the iterator from the dictionary and using the iterator:

In [22]:
d_iter = iter(d)
for k in d_iter:
    print(k)

p
y
t
h
o
n


So, you may very well be asking yourself whether we should iterate keys using the dictionaries iterator, or using the key view? After all it seems to do the same thing...

And yes, either one would be just fine for iterating over the keys of a dictionary. Let's just make sure the performance is about the same:

In [23]:
from timeit import timeit
from random import randint

d = {k: randint(0, 100) for k in range(10_000)}
keys = d.keys()

def iter_direct(d):
    for k in d:
        pass
    
def iter_view(d):
    for k in d.keys():
        pass

def iter_view_direct(view):
    for k in view:
        pass
    
print(timeit('iter_direct(d)', globals=globals(), number=20_000))
print(timeit('iter_view(d)', globals=globals(), number=20_000))
print(timeit('iter_view_direct(keys)', globals=globals(), number=20_000))

1.857292921980843
1.8094384070136584
1.8164116419502534


As you can see, unless you are re-creating a new view object every time, the performance difference between iterating via the dictionary's iterator and the view's iterator is about the same. [In fact, it's the same iterator in both cases!] 
But since there is really no need to re-create a view once it's been created (since is is dynamic), the overhead of creating the `keys` view is a one-time hit.
And a `keys` view provides far more functionality than just iteration - as we know it behaves like a set - so if you need to perform set operations on the keys you'll need to use the `keys` view.

#### Iterating over keys and values

As we saw, we can use the `.items()` view to iterate over both the keys and values of a dictionary.

In [24]:
d = {'a': 1, 'b': 2, 'c': 3}

In [25]:
for k, v in d.items():
    print(k, v)

a 1
b 2
c 3


You might be tempted to do it this way as well:

In [26]:
for k in d:
    print(k, d[k])

a 1
b 2
c 3


But this is quite inefficient!
Let's try some timings.

In [27]:
d = {k: randint(0, 100) for k in range(10_000)}
items = d.items()

def iterate_view(view):
    for k, v in view:
        pass
    
def iterate_clunky(d):
    for k in d:
        d[k]
        
print(timeit('iterate_view(items)', globals=globals(), number=5_000))
print(timeit('iterate_clunky(d)', globals=globals(), number=5_000))

0.8359718360006809
1.3389352719532326


As you can see, it is substantially slower to iterate over both the keys and the values of the dictionary using the second approach. This is because in the second approach, we have to perform a lot of dictionary lookups - while lookups are particularlay efficient in Python, they are slower than not doing a lookup at all!

#### Iterating a dictionary while mutating keys

As we mentioned earlier, we cannot mutate a dictionary's keys while iterating over it:

Let's see an example of this:

In [28]:
d = {'a': 1, 'b': 2, 'c': 3}
for k, v in d.items():
    print(k, v ** 2)
    del d[k]

a 1


RuntimeError: dictionary changed size during iteration

One way to solve this is to create a static list of all the keys, and iterate over that instead:

In [29]:
d = {'a': 1, 'b': 2, 'c': 3}
keys = list(d.keys())
print(keys)

['a', 'b', 'c']


In [30]:
for k in keys:
    value = d.pop(k)
    print(f'{value} ** 2 = {value ** 2}')

1 ** 2 = 1
2 ** 2 = 4
3 ** 2 = 9


In [31]:
d

{}

Another way would be to use the `popitem` method. We just need to know how many times we can call `popitem`, or catch the `KeyError` exception when it occurs:

In [32]:
d = {'a':1, 'b':2, 'c':3}
for _ in range(len(d)):
    key, value = d.popitem()
    print(key, value, value**2)

c 3 9
b 2 4
a 1 1


Or we can use a `while` loop:

In [33]:
d = {'a':1, 'b':2, 'c':3}
while len(d) > 0:
    key, value = d.popitem()
    print(key, value, value**2)

c 3 9
b 2 4
a 1 1


Or we can simply keep iterating indefinitely until a `KeyError` exception occurs:

In [34]:
d = {'a':1, 'b':2, 'c':3}
while True:
    try:
        key, value = d.popitem()
    except KeyError:
        break
    else:
        print(key, value, value**2)

c 3 9
b 2 4
a 1 1
